In [1]:
import os 
import numpy as np
import pandas as pd
from numpy import ndarray
from nsddatapaper_rsa.utils.nsd_get_data import get_conditions, get_labels, get_betas
from nsddatapaper_rsa.utils.utils import category_dict, mds, average_over_conditions
from utils.utils import *
from utils.kabsch2D import *
from nsd_access import NSDAccess 
from matplotlib import cm 
import scprep
import matplotlib.pyplot as plt
import nibabel as nib
from scipy.spatial.distance import pdist

True


In [2]:
# Check MDS for 40 sessions

In [2]:
n_jobs = -1
n_sessions = 32 # right so turns out this is fucked and different subject have different sessions number 
n_subjects = 8

nsda = NSDAccess(nsd_dir)

outpath = os.path.join(betas_dir, 'roi_analyses')
if not os.path.exists(outpath):
    os.makedirs(outpath)

targetspace = 'nativesurface'  # need to ask Ben: why nativesurface instead of fsaverage? Seems to make sens esince we're casting on the cortical surface but need to be sure

# I found this in the paper repo -> mainfigures -> SCIENCE.RSA
lh_file = os.path.join(proj_dir, 'lh.highlevelvisual.mgz')
rh_file = os.path.join(proj_dir, 'rh.highlevelvisual.mgz')


# load them
maskdata_lh = nib.load(lh_file).get_fdata().squeeze()      
maskdata_rh = nib.load(rh_file).get_fdata().squeeze()

maskdata = np.hstack((maskdata_lh, maskdata_rh))   # try to load betas with this mask maybe? 

ROIS = {1: 'pVTC', 2: 'aVTC', 3: 'v1', 4: 'v2', 5: 'v3'} # not used

roi_names = ['pVTC', 'aVTC', 'v1', 'v2', 'v3']

# sessions

# subjects
subs = ['subj0{}'.format(x+1) for x in range(n_subjects)]

In [3]:
outpath = os.path.join(proj_dir, 'rsa', 'roi_analyses')

In [5]:
# get mask 
maskdata_long = np.load('/media/harveylab/STORAGE1 nA/luis-thesis/spatial_neural_responses/data/nsddata/freesurfer/subj06/label/customrois/subj06.testrois.npy', allow_pickle=True)
maskdata_long_bool = (maskdata_long > 0)
maskdata_long_bool

array([ True,  True,  True, ..., False, False, False])

In [6]:
conditions = get_conditions(nsd_dir, subs[5], n_sessions)

		sub: subj06 fetching condition trials in session: 1
		sub: subj06 fetching condition trials in session: 2
		sub: subj06 fetching condition trials in session: 3
		sub: subj06 fetching condition trials in session: 4
		sub: subj06 fetching condition trials in session: 5
		sub: subj06 fetching condition trials in session: 6
		sub: subj06 fetching condition trials in session: 7
		sub: subj06 fetching condition trials in session: 8
		sub: subj06 fetching condition trials in session: 9
		sub: subj06 fetching condition trials in session: 10
		sub: subj06 fetching condition trials in session: 11
		sub: subj06 fetching condition trials in session: 12
		sub: subj06 fetching condition trials in session: 13
		sub: subj06 fetching condition trials in session: 14
		sub: subj06 fetching condition trials in session: 15
		sub: subj06 fetching condition trials in session: 16
		sub: subj06 fetching condition trials in session: 17
		sub: subj06 fetching condition trials in session: 18
		sub: subj06 fetch

In [7]:
conditions = np.asarray(conditions).ravel()
conditions_bool = [
    True if np.sum(conditions == x) == 3 else False for x in conditions]

conditions_sampled = conditions[conditions_bool]

# find the subject's unique condition list (sample pool)
sample = np.unique(conditions[conditions_bool])
assert sample.shape[0] == 10000

AssertionError: 

In [8]:
for i in range(len(subs)):
    print(subs[i])

subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08


In [6]:
sessions = [40, 40, 32, 30, 40, 32, 40, 30]
for i in range(len(subs)):

    maskdata_long = np.load(f'/media/harveylab/STORAGE1_NA/luis-thesis/spatial_neural_responses/data/nsddata/freesurfer/subj0{i+1}/label/customrois/subj0{i+1}.testrois.npy', allow_pickle=True)
    maskdata_long_bool = (maskdata_long > 0)
    maskdata_long_bool
    print(maskdata_long_bool.shape)
    
    conditions = get_conditions(nsd_dir, subs[i], sessions[i] )

    conditions = np.asarray(conditions).ravel()
    conditions_bool = [
        True if np.sum(conditions == x) == 3 else False for x in conditions]

    conditions_sampled = conditions[conditions_bool]

    # find the subject's unique condition list (sample pool)
    sample = np.unique(conditions[conditions_bool])
    
    betas_mean_file = os.path.join(outpath, f'{subs[i]}_betas_list_{targetspace}_averaged.npy')   # try with now averaging 
    if not os.path.exists(betas_mean_file):
        betas_mean = get_betas(
            nsd_dir, 
            subs[i],
            sessions[i],
            mask=maskdata_long_bool,
            targetspace=targetspace
        )
        print(f'concatenating betas for {subs[i]}')
      #  print(betas_mean.shape)
        betas_mean = np.concatenate(betas_mean, axis=1).astype(np.float32)

        print(f'Now averaging them')
        betas_mean = average_over_conditions(
            betas_mean,
            conditions,
            conditions_sampled
        ).astype(np.float32)

        if np.isnan(betas_mean).any():    # drop the full row if there's some non (so drop the whole voxel) 
            betas_mean = betas_mean[~np.isnan(betas_mean).any(axis=1), :]    # This is fine 
     #   print(betas_mean.shape)
        print(f'Saving conditions averaged betas')
        np.save(betas_mean_file, betas_mean)

    else:
        print(f'loading betas for {subs[i]}')
        betas_mean = np.load(betas_mean_file, allow_pickle=True)
        print(f'shape of betas {betas_mean.shape}')

(453622,)
		sub: subj01 fetching condition trials in session: 1
		sub: subj01 fetching condition trials in session: 2
		sub: subj01 fetching condition trials in session: 3
		sub: subj01 fetching condition trials in session: 4
		sub: subj01 fetching condition trials in session: 5
		sub: subj01 fetching condition trials in session: 6
		sub: subj01 fetching condition trials in session: 7
		sub: subj01 fetching condition trials in session: 8
		sub: subj01 fetching condition trials in session: 9
		sub: subj01 fetching condition trials in session: 10
		sub: subj01 fetching condition trials in session: 11
		sub: subj01 fetching condition trials in session: 12
		sub: subj01 fetching condition trials in session: 13
		sub: subj01 fetching condition trials in session: 14
		sub: subj01 fetching condition trials in session: 15
		sub: subj01 fetching condition trials in session: 16
		sub: subj01 fetching condition trials in session: 17
		sub: subj01 fetching condition trials in session: 18
		sub: su

/media/harveylab/STORAGE1_NA/stan-thesis/src/nsddatapaper_rsa/utils/utils.py:144: RuntimeWarning: Mean of empty slice
  avg_data[:,j] = np.nanmean(data[:, conditions_bool], axis=1)


Saving conditions averaged betas
(399162,)
		sub: subj07 fetching condition trials in session: 1
		sub: subj07 fetching condition trials in session: 2
		sub: subj07 fetching condition trials in session: 3
		sub: subj07 fetching condition trials in session: 4
		sub: subj07 fetching condition trials in session: 5
		sub: subj07 fetching condition trials in session: 6
		sub: subj07 fetching condition trials in session: 7
		sub: subj07 fetching condition trials in session: 8
		sub: subj07 fetching condition trials in session: 9
		sub: subj07 fetching condition trials in session: 10
		sub: subj07 fetching condition trials in session: 11
		sub: subj07 fetching condition trials in session: 12
		sub: subj07 fetching condition trials in session: 13
		sub: subj07 fetching condition trials in session: 14
		sub: subj07 fetching condition trials in session: 15
		sub: subj07 fetching condition trials in session: 16
		sub: subj07 fetching condition trials in session: 17
		sub: subj07 fetching conditio

/media/harveylab/STORAGE1_NA/stan-thesis/src/nsddatapaper_rsa/utils/utils.py:144: RuntimeWarning: Mean of empty slice
  avg_data[:,j] = np.nanmean(data[:, conditions_bool], axis=1)


Saving conditions averaged betas


In [93]:
path = os.path.join(outpath, f'subj06_betas_list_{targetspace}_averaged.npy') 
betas_mean = np.load(path)
coords = np.argwhere(np.isnan(betas_mean)) #is it image or voxel dimension?

In [95]:
betas_mean_no_nan = betas_mean[~np.isnan(betas_mean).any(axis=1), :]
betas_mean_no_nan.shape

(38173, 6234)

In [ ]:
# save full ROI
betas_mean = np.load('subj01_betas_list_fsaverage_averaged.npy', allow_pickle=True)

for roi in range(1, 6):
    mask_name = rois[roi]
    rdm_file = os.path.join(outpath, f'{subs[0]}_{mask_name}_fullrdm_correlation.npy'
                           )

    if not os.path.exists(rdm_file):

        #array of mask vertices
        vs_mask = maskdata == roi
        print(f'working on ROI: {mask_name}')

        masked_betas = betas_mean[vs_mask, :]

        good_vox = [
            True if np.sum(np.isnan(x)) == 0 else False for x in masked_betas]

        if np.sum(good_vox) != len(good_vox):
            print(f'found some NaN for ROI: {mask_name} - {subs[0]}')

        # prepare for correlation distance
        X = masked_betas.T

        print(f'computing RDM for roi: {mask_name}')
        rdm = pdist(X, metric='correlation')

        if np.any(np.isnan(rdm)):
            raise ValueError

        print(f'saving full rdm for {mask_name} : {subs[0]}')
        np.save(rdm_file, rdm)